# Forward/Inverse Kinematics Examples

Kevin Walchko
Created: 10 July 2017

---

...

## References

- [Wikipedia modified DH](https://en.wikipedia.org/wiki/Denavit%E2%80%93Hartenberg_parameters)
- [darpa robot challenge](https://www.youtube.com/watch?v=diaZFIUBMBQ)
- [darpa robot fails](https://youtu.be/wX0KagJ1du8)

## Setup

In [1]:
%matplotlib inline

In [44]:
from __future__ import print_function
from __future__ import division
import numpy as np
from sympy import symbols, sin, cos, pi, simplify, trigsimp
from math import radians as d2r
from math import degrees as r2d
from math import atan2, sqrt, acos, fabs
from pprint import pprint

# Denavit-Hartenberg (DH)

Development of the DH forward kinematics for a robot. We will use the DH method to develop the symbolic equations and use python to simplify them.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d8/DHParameter.png/609px-DHParameter.png)



# Kinematics

Robot arms and legs are hard to control (lots of math), but required for most robotic applications.

## Coordinate Frames

To help, we are going to follow a methodical way to assign coordinate frames to a robot. An example (sort of) is shown below:

![](dh_pics/kuka-kr270.png)

The KR270 has 6 joints and is a standard industrial type robot for building cars or what not.

## Numerical Forward Kinematics

---

The following class follows the modified [DH convention](https://en.wikipedia.org/wiki/Denavit%E2%80%93Hartenberg_parameters). Where:

| Parameter | Definition|
|-----------|-----------|
| $\alpha_i$   | angle about common normal, from old z axis to new z axis (twist)|
| $d_i$        | offset along previous z to the common normal |
| $\theta_i$   | angle about previous z, from old x to new x |
| $a_i$        | length of the common normal. Assuming a revolute joint, this is the radius about previous z. |

Now the transformation matrix ($T$), using modified DH parameters, is given by:

$$ T = R_x(\alpha_{n-1}) D_x(a_{n-1}) R_z(\theta_n) D_z(d_n) $$

We can then turn this into a python class as follows:

In [102]:
class mDH(object):
    """
    This uses the modified DH parameters
    see Craig, eqn (3.6)
    """
    def __init__(self):
        pass

    def fk(self, params):
        t = np.eye(4)
        for p in params:
            t = t.dot(self.makeT(*p))
        return t

    def makeT(self, a, alpha, d, theta):
#         alpha = d2r(alpha)
#         theta = d2r(theta)
        return np.array([  # classic DH
            [           cos(theta),           -sin(theta),           0,             a],
            [sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -d*sin(alpha)],
            [sin(theta)*sin(alpha), cos(theta)*sin(alpha),  cos(alpha),  d*cos(alpha)],
            [                    0,                     0,           0,             1]
        ])


def printT(tt):
    R = tt[0:3,0:3]
    D = tt[0:3, 3]
    print('-'*30)
    print('Position:')
    print('  x:', D[0])
    print('  y:', D[1])
    print('  z:', D[2])
    # R(n, o, a)
    print('-'*30)
    print('Orientation')
    print('  nx:', R[0,0])
    print('  ny:', R[0,1])
    print('  nz:', R[0,2])
    print('')
    print('  ox:', R[1,0])
    print('  oy:', R[1,1])
    print('  oz:', R[1,2])
    print('')
    print('  ax:', R[2,0])
    print('  ay:', R[2,1])
    print('  az:', R[2,2])

    
def simplifyT(tt):
    
    for i, row in enumerate(tt):
        for j, col in enumerate(row):
            tt[i,j] = simplify(col)
    return tt

## Symbolically Solving Forward Kinematics

In [31]:
t1, t2, t3 = symbols('t1 t2 t3')
L1, L2, L3 = symbols('L1 L2 L3')

In [4]:
def eval(f, inputs):
    """
    This allows you to simplify the trigonomic mess that kinematics can
    create and also substitute in some inputs in the process
    """
    c = []
    for i in f:
        r = []
        for j in i:
            tmp = j
#             tmp = j.subs(inputs)
            # substitute in any in puts if entered
#             tmp = tmp.evalf()
            # use python symbolic toolbox to simplify the trig mess above 
            tmp = simplify(tmp)
            r.append(tmp)
        c.append(r)
    return np.array(c)

The parameters are:

| i |$a_i$        | $\alpha_i$   | $d_i$   | $theta_i$  |
|---|-------------|--------------|---------|------------|
| 1 | 0           | 0            | 0       | $\theta_1$ |
| 2 | $L_1$       | 0            | 0       | $\theta_2$ |
| 3 | $L_2$       | 0            | 0       | $\theta_3$ |


In [55]:
t1, t2, t3 = symbols('t1 t2 t3')
L1, L2, L3 = symbols('L1 L2 L3')
# a, alpha, d, theta
params = [
    [ 0, 0, 0, t1],  # DH parameters for the first link (i.e., the first row in the table above)
    [L1, 0, 0, t2],  # 2nd link
    [L2, 0, 0, t3]   # 3rd link
]
dh = mDH()
t = dh.fk(params)
t = eval(t,[])

In [56]:
R = t[0:3,0:3]
print('Rotation Matrix:')
pprint(R)
D = t[0:3, 3]
print('Position:')
print('  x:', D[0])
print('  y:', D[1])
print('  z:', D[2])

Rotation Matrix:
array([[1.0*cos(t1 + t2 + t3), -1.0*sin(t1 + t2 + t3), 0],
       [1.0*sin(t1 + t2 + t3), 1.0*cos(t1 + t2 + t3), 0],
       [0, 0, 1.00000000000000]], dtype=object)
Position:
  x: 1.0*L1*cos(t1) + 1.0*L2*cos(t1 + t2)
  y: 1.0*L1*sin(t1) + 1.0*L2*sin(t1 + t2)
  z: 0


In [25]:
# now, if we want better performance, you can just use the above solution
# for our forward kinematics instead of building it each time we need it
# using the mDH class (which is a lot more computational)
def fk(t1, t2, d, L1, L2, L3):
    t1 = d2r(t1)  # python uses radians, and typicall we use degrees, so fix it
    t2 = d2r(t2)
    return np.array([
        1.0*L1*cos(t1) + 1.0*L2*cos(t1 + t2) + 1.0*L3*cos(t1 + t2),
        1.0*L1*sin(t1) + 1.0*L2*sin(t1 + t2) + 1.0*L3*sin(t1 + t2),
        0
    ])

# Inverse Kinematics


---


Assume the solution is this:

$$
\begin{bmatrix}
    c_{\gamma} & -s_{\gamma} & 0 & x \\
    s_{\gamma} & c_{\gamma} & 0 & y \\
    0 & 0 & 1 & 0 \\
    0 & 0 & 0 & 1 
\end{bmatrix}
$$

Note, that there is simplifications here to make reading/writing these equations easier, such as:

$$
\cos(\theta_1) = c_1 \\
\sin(\theta_1 + \theta_1) = s_{12}
$$

Now set the above matrix equal to it, (and following along in Craig, p 110) we get:

$$
c_{\gamma} = c_{123} \\
s_{\gamma} = s_{123} \\
x = L1*c_1+L2*c_{12} \\
y = L1*s_1+L1*s_{12}
$$

In [110]:
def trans(tt):
    # calculates the inverse of a tranformation matrix
    r = tt[0:3,0:3]
    d = tt[0:3, 3]
#     print('r', r.transpose())
    ret = np.eye(4, dtype=object)
    ret[0:3,0:3] = r.transpose()
#     rt = r.transpose()
#     for i in range(3):
#         for j in range(3):
#             ret[i,j] = rt[i,j]
    ret[0:3,3] = -r.transpose().dot(d)
    return ret

In [88]:
print(t)
print('-'*20)
print(trans(t))

[[1.0*cos(t1 + t2 + t3) -1.0*sin(t1 + t2 + t3) 0
  1.0*L1*cos(t1) + 1.0*L2*cos(t1 + t2)]
 [1.0*sin(t1 + t2 + t3) 1.0*cos(t1 + t2 + t3) 0
  1.0*L1*sin(t1) + 1.0*L2*sin(t1 + t2)]
 [0 0 1.00000000000000 0]
 [0 0 0 1.00000000000000]]
--------------------
[[1.0*cos(t1 + t2 + t3) 1.0*sin(t1 + t2 + t3) 0
  -1.0*(1.0*L1*sin(t1) + 1.0*L2*sin(t1 + t2))*sin(t1 + t2 + t3) - 1.0*(1.0*L1*cos(t1) + 1.0*L2*cos(t1 + t2))*cos(t1 + t2 + t3)]
 [-1.0*sin(t1 + t2 + t3) 1.0*cos(t1 + t2 + t3) 0
  -1.0*(1.0*L1*sin(t1) + 1.0*L2*sin(t1 + t2))*cos(t1 + t2 + t3) + 1.0*(1.0*L1*cos(t1) + 1.0*L2*cos(t1 + t2))*sin(t1 + t2 + t3)]
 [0 0 1.00000000000000 0]
 [0 0 0 1]]



-----------

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.